<a href="https://colab.research.google.com/github/bofeng2268/qm2/blob/main/blue_badge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pandas
!pip install geopandas
!pip install plotly

In [3]:
import pandas as pd
import plotly
import plotly.express as px
import matplotlib.pyplot as plt
import geopandas as gpd
import requests
import json


In [4]:
# Paths to the files with Raw GitHub URLs
disability_file_path = 'https://raw.githubusercontent.com/bofeng2268/qm2/main/Phenomena/Local%20Authority%20Disability/TS038-2021-3-filtered-2024-01-01T16-28-10Z.csv'
topojson_file_path = 'https://raw.githubusercontent.com/bofeng2268/qm2/main/Phenomena/Local%20Authority%20Disability/topo_lad.json'
blue_badges_file_path = 'https://raw.githubusercontent.com/bofeng2268/qm2/main/BlueBadge/Blue_badges_held.csv'

# Load the CSV files
disability_data = pd.read_csv(disability_file_path)
blue_badges_held = pd.read_csv(blue_badges_file_path, delimiter=';')

# Load the TopoJSON file from the URL
response = requests.get(topojson_file_path)
topojson_data = response.json()
gdf = gpd.read_file(json.dumps(topojson_data), layer='lad')

In [5]:
# Filter and aggregate the disability data
eng_disability_data = disability_data[disability_data['Lower tier local authorities Code'].str.startswith('E')]
filtered_disability_data = eng_disability_data[eng_disability_data['Disability (5 categories) Code'].isin([1, 2])]
aggregated_disability_data = filtered_disability_data.groupby('Lower tier local authorities')['Observation'].sum().reset_index()

print(aggregated_disability_data)


    Lower tier local authorities  Observation
0                           Adur        12577
1                      Allerdale        19205
2                   Amber Valley        25066
3                           Arun        32481
4                       Ashfield        27809
..                           ...          ...
304                     Worthing        21303
305                     Wychavon        22955
306                         Wyre        24771
307                  Wyre Forest        20459
308                         York        34592

[309 rows x 2 columns]


In [6]:
#Filter blue badge data

blue_badges_narrowed =  blue_badges_held[['ONS Code', 'Local Authority', 'Total valid Blue Badges held by organisations and individuals']]
filtered_blue_badges = blue_badges_narrowed[~blue_badges_narrowed['Local Authority'].str.contains('All')]

print(filtered_blue_badges)


      ONS Code  Local Authority  \
2    E06000047    County Durham   
3    E06000005       Darlington   
4    E06000001       Hartlepool   
5    E06000002    Middlesbrough   
6    E06000057   Northumberland   
..         ...              ...   
163  E06000054        Wiltshire   
164  E10000008            Devon   
165  E06000059           Dorset   
166  E10000013  Gloucestershire   
167  E10000027        Somerset    

    Total valid Blue Badges held by organisations and individuals  
2                                               27.884             
3                                                5.945             
4                                                5.530             
5                                                6.546             
6                                               16.242             
..                                                 ...             
163                                             23.267             
164                                

In [31]:
merged_data = pd.merge(filtered_blue_badges, aggregated_disability_data, left_on='Local Authority', right_on='Lower tier local authorities', how='inner')


# Convert columns to numeric
merged_data['Total valid Blue Badges held by organisations and individuals'] = pd.to_numeric(merged_data['Total valid Blue Badges held by organisations and individuals'])
merged_data['Observation'] = pd.to_numeric(merged_data['Observation'])

# Calculate the ratio
merged_data['ratio'] = merged_data['Total valid Blue Badges held by organisations and individuals'] / merged_data['Observation']
#do ratio of values blue badge/total disabled pop

# Merge the GeoDataFrame with the merged data
final_merged_data = gdf.merge(merged_data, left_on='LAD13NM', right_on='Local Authority')
print(final_merged_data)


            id    LAD13CD LAD13CDO               LAD13NM LAD13NMW  \
0    E06000001  E06000001     00EB            Hartlepool     None   
1    E06000002  E06000002     00EC         Middlesbrough     None   
2    E06000003  E06000003     00EE  Redcar and Cleveland     None   
3    E06000004  E06000004     00EF      Stockton-on-Tees     None   
4    E06000005  E06000005     00EH            Darlington     None   
..         ...        ...      ...                   ...      ...   
98   E09000029  E09000029     00BF                Sutton     None   
99   E09000030  E09000030     00BG         Tower Hamlets     None   
100  E09000031  E09000031     00BH        Waltham Forest     None   
101  E09000032  E09000032     00BJ            Wandsworth     None   
102  E09000033  E09000033     00BK           Westminster     None   

                                              geometry   ONS Code  \
0    MULTIPOLYGON (((-1.24244 54.72219, -1.24162 54...  E06000001   
1    MULTIPOLYGON (((-1.19744 54.

In [35]:
"""
Cloropleth map of ratio:
"""
final_merged_data.set_index('id', inplace=True)
# Create a choropleth map using Plotly
fig = px.choropleth_mapbox(final_merged_data,
                           geojson=final_merged_data.geometry,
                           locations=final_merged_data.index,
                           color='ratio',
                           hover_name="Local Authority",
                           mapbox_style="carto-positron",
                           zoom=5, center = {"lat": 52.3555, "lon": -1.1743},
                           opacity=0.5)

# Update layout
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# Display the map
fig.show()